# Ag Analytics-FederalCropInsuranceQuoter-Demo


To calculate insurance premiums for Yield Protection, Revenue Protection, and Revenue Protection with Harvest Price exclusion, along with their area coverage alternatives.


In [ ]:
import requests
import json
import time
from pandas.io.json import json_normalize
import pandas as pd
from collections import defaultdict
%autosave 0

### API details

####Request URL:
https://ag-analytics.azure-api.net/FederalCropInsuranceQuoter/post

####Request body:

***'FIPS' :*** The FIPS code of a county (e.g. 17019). 

***'CropCode':*** The code for a crop (e.g. corn's crop code is 41) from table A00420 of RMA ADM files. 
***'Type' : ***The code for the type of a crop (e.g. corn type grain is 16) from RMA ADM files. 
***'Practice': ***The code for the practice type of a crop (e.g. non-irrigated is 3) from RMA ADM files. Call the Practice API for available for that cfrtop 
***'PreventedPlanning' :*** An integer in the range [0, 2]. 0 = Standard, 1 = Plus 5%, 2 = Plus 1. 
***'UseTAYield' : ***int An integer in the range [0, 1]. 1 = Use Trend-Adjusted Yield, 0 = do not use Trend-Adjusted Yield. 
***'SharePercentage' : ***double A float indicating the insured share percent, in the range [0.001, 1]. 
***'TrendAdjustedYield' :*** double A double specifying the Trend-Adjusted Yield. 
***'Acres' : ***double A double specifying the acreage. 
***'Year' :*** int An integer specifying the year that the calculation should take place for. 
***'APHYield' :*** double A double specifying the Actual Production History Yield. 
***'Price' :*** double A double specifying the projected crop price. 
***'Volatility' :*** double A double specifying the volatility of the crop. 
***‘ReturnParameters’ :*** int If value is 1, parameters for the steps of the premium calculation are included in the response. If value is 0, parameters from the premium calculation are not included in the response. 
***‘HighRiskCode’:***	string Subcounty high risk code that the insured field is located in and is relevant to the specified crop, practice, and type (e.g. ‘AAA’). 



In order to get Ocp-Apim-Subscription-Key, please click on this link https://ag-analytics.org/developer/Session/SignInFromPayment

### Request Parameters

In [ ]:
requestbody={'FIPS':17081, 'CropCode':41, 'Type':16, 'Practice':3, 'PreventedPlanting':0, 'UseTAYield':1, 'UsePerAcre':1, 'SharePercentage':1.00, 'TrendAdjustedYield':129.88, 'Acres':44.56999969, 'Year':2019, 'APHYield':129.88, 'Price':4.00, 'Volatility':0.15, 'IncludeAdminFee':0.0}

headers={'Content-Type':'application/json',
'Ocp-Apim-Subscription-Key': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'}
           

###  API Function

In [ ]:
def post_crop_insurance_quoter(requestbody,headers):
    try:
        url = "https://ag-analytics.azure-api.net/FederalCropInsuranceQuoter/post"
     
        response = (requests.post(url,data=json.dumps(requestbody),headers=headers).text)
        
        
        response_json = json.loads(response)
        
        print(response_json)
        time.sleep(0.5)
        return response_json
    
    except Exception as e:
        raise e

### Calling API Function and Displaying Response

In [ ]:
crop_insurance=post_crop_insurance_quoter(requestbody,headers)

In [ ]:
df=defaultdict(list)


for i in range(3):
    for value,u in zip(['Revenue Protection','Revenue Protection with Harvest Price Exclusion','Yield Protection'],['Optional','Basic','Enterprise']):
        
        df['Product Type'].append(value)
        df['Unit'].append(u)
    
coverage=['50%','55%','60%','65%','70%','75%','80%','85%']    


for k in range(9):
    for i,j in enumerate(coverage):
        
        df[j+'Premium perAcre'].append(crop_insurance['Premium'][i][k])
        df[j+'Premium totalAcre'].append(crop_insurance['PremiumAllAcres'][i][k])
        df[j+'Liability total Acre'].append(crop_insurance['Liability'][i][0])

    

df=pd.DataFrame.from_dict(df)
df
